<a href="https://colab.research.google.com/github/javisnes/Analisis-de-Abandono-de-Clientes/blob/main/An%C3%A1lisis_de_Abandono_de_Clientes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import random

# Generar datos base
np.random.seed(42)
n = 1200
data = {
    'ID_Cliente': [f'CLI-{1000+i}' for i in range(n)],
    'Edad': [random.randint(18, 85) if random.random() > 0.05 else np.nan for i in range(n)],
    'Tipo_Contrato': [random.choice(['Mensual', 'Anual', 'Mensuall', 'Anuall']) for i in range(n)],
    'Cargos_Mensuales': [random.uniform(20, 120) for i in range(n)],
    'Meses_Antiguedad': [random.randint(1, 72) if random.random() > 0.03 else -5 for i in range(n)],
    'Abandono': [random.choice(['Si', 'No']) for i in range(n)]
}

df_churn = pd.DataFrame(data)
# Agregar duplicados
df_churn = pd.concat([df_churn, df_churn.sample(n=30)]).reset_index(drop=True)

print("✅ Dataset 'Telecom_Churn_Dirty.csv' creado con 1230 filas.")

✅ Dataset 'Telecom_Churn_Dirty.csv' creado con 1230 filas.
